Neo4j 지식 그래프를 기반으로, 사용자의 자연어 질문에 대해 정확하고 풍부한 답변을 제공하는 GraphRAG 시스템을 구현합니다.

단순한 키워드 매칭이나 벡터 검색의 한계를 넘어, 그래프의 구조적 정보(관계)와 벡터의 의미적 정보(유사도)를 결합하여 최적의 검색 결과를 도출합니다.


시스템은 질문의 유형에 따라 세 가지 검색 전략을 통합하여 사용합니다:

1. **[Vector]** Content 노드 기반 벡터 검색 - 내용 유사도 기반 검색
2. **[VectorCypher]** - 복합 검색

3. **[Text2Cypher]** 자연어를 Cypher로 변환한 구조적 검색


In [2]:
!pip install langchain langchain-community langchain-openai neo4j python-dotenv tiktoken neo4j_graphrag

In [3]:
from neo4j import GraphDatabase, basic_auth
NEO4J_URI = "neo4j+s://579bb57f.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PW   = os.getenv("NEO4J_PW")

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=basic_auth(NEO4J_USER, NEO4J_PW)
)

In [4]:
import os
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from neo4j_graphrag.embeddings import Embedder
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.indexes import create_vector_index

MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
EMBED_DIM = 384

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## vector

In [5]:
class LocalEmbedder(Embedder):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text: str) -> list[float]:
        return self.model.encode(text, normalize_embeddings=True).tolist()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()
embedder = LocalEmbedder(MODEL_NAME)

In [8]:
def fetch_contents_needing_embedding(tx, limit=200):
    q = """
    MATCH (c:Content)
    WHERE (c.text IS NOT NULL AND c.text <> "")
       OR (c.detail IS NOT NULL AND c.detail <> "")
      AND (c.embedding IS NULL OR c.embedding_model <> $model)
    RETURN c.content_id AS content_id, c.text AS text, c.name AS name, c.detail AS detail
    LIMIT $limit
    """
    return list(tx.run(q, limit=limit, model=MODEL_NAME))

def update_content_embeddings(tx, rows):
    q = """
    UNWIND $rows AS row
    MATCH (c:Content {content_id: row.content_id})
    SET c.embedding = row.embedding,
        c.text = row.text,
        c.embedding_model = $model,
        c.embedding_dim = $dim
    """
    tx.run(q, rows=rows, model=MODEL_NAME, dim=EMBED_DIM)

def embed_all_contents(batch_size=100, max_batches=100):
    print(f"Start Embedding with model: {MODEL_NAME}...")
    total = 0
    with driver.session() as session:
        for i in range(max_batches):
            records = session.execute_read(fetch_contents_needing_embedding, batch_size)
            if not records:
                print("모든 데이터 임베딩 완료.")
                break

            processed_rows = []
            texts_to_embed = []

            for r in records:
                text_content = r["text"]
                if not text_content and r["detail"]:
                    text_content = f"[{r['name']}] {r['detail']}"

                texts_to_embed.append(text_content)
                processed_rows.append({"content_id": r["content_id"], "text": text_content})

            # 벡터 생성
            embeddings = embedder.embed_documents(texts_to_embed)

            rows = []
            for row_data, vec in zip(processed_rows, embeddings):
                row_data["embedding"] = vec
                rows.append(row_data)

            session.execute_write(update_content_embeddings, rows)

            total += len(rows)
            print(f"[Batch {i+1}] {len(rows)}개 저장 (Total: {total})")

embed_all_contents()


### 벡터 인덱스 생성
VECTOR_INDEX_NAME = "vector_index"

try:
    # 기존 인덱스가 있다면 삭제 후 재생성 (차원 충돌 방지)
    with driver.session() as session:
        session.run(f"DROP INDEX {VECTOR_INDEX_NAME} IF EXISTS")

    create_vector_index(
        driver,
        VECTOR_INDEX_NAME,
        "Content",
        "embedding",
        EMBED_DIM,
        "cosine"
    )
    print(f"[OK] 인덱스({VECTOR_INDEX_NAME}) 생성 완료")
except Exception as e:
    print(f"인덱스 오류: {e}")



Start Embedding with model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2...


[Batch 1] 100개 저장 (Total: 100)
[Batch 2] 100개 저장 (Total: 200)
[Batch 3] 100개 저장 (Total: 300)
[Batch 4] 100개 저장 (Total: 400)
[Batch 5] 100개 저장 (Total: 500)
[Batch 6] 100개 저장 (Total: 600)
[Batch 7] 100개 저장 (Total: 700)
[Batch 8] 100개 저장 (Total: 800)
[Batch 9] 100개 저장 (Total: 900)
[Batch 10] 100개 저장 (Total: 1000)
[Batch 11] 100개 저장 (Total: 1100)
[Batch 12] 100개 저장 (Total: 1200)
[Batch 13] 100개 저장 (Total: 1300)
[Batch 14] 100개 저장 (Total: 1400)
[Batch 15] 100개 저장 (Total: 1500)
[Batch 16] 100개 저장 (Total: 1600)
[Batch 17] 100개 저장 (Total: 1700)
[Batch 18] 100개 저장 (Total: 1800)
[Batch 19] 100개 저장 (Total: 1900)
[Batch 20] 100개 저장 (Total: 2000)
[Batch 21] 100개 저장 (Total: 2100)
[Batch 22] 100개 저장 (Total: 2200)
[Batch 23] 100개 저장 (Total: 2300)
[Batch 24] 100개 저장 (Total: 2400)
[Batch 25] 100개 저장 (Total: 2500)
[Batch 26] 100개 저장 (Total: 2600)
[Batch 27] 100개 저장 (Total: 2700)
[Batch 28] 100개 저장 (Total: 2800)
[Batch 29] 100개 저장 (Total: 2900)
[Batch 30] 100개 저장 (Total: 3000)
[Batch 31] 100개 저장 (Total: 3

In [ ]:
retriever = VectorRetriever(
        driver=driver,
        index_name=VECTOR_INDEX_NAME,
        embedder=embedder,
        return_properties=["text", "content_id"]
    )

In [12]:
def check_vector_search(query, top_k=3):
    print(f"질문: {query}\n")

    # 검색 수행
    results = retriever.search(query_text=query, top_k=top_k)

    # 결과 출력
    for i, item in enumerate(results.items):

        content = item.content
        score = item.metadata.get('score', 0.0)

        if isinstance(content, str):
            try:
                import ast
                content = ast.literal_eval(content)
            except:
                pass

        content_id = content.get('content_id', 'Unknown') if isinstance(content, dict) else 'Unknown'
        text_preview = content.get('text', '') if isinstance(content, dict) else str(content)

        print(f"[Rank {i+1}]")
        print(f"  - Score      : {score:.4f}")
        print(f"  - Content ID : {content_id}")
        print(f"  - Text       : {text_preview}...")
        print("-" * 50)

# 3. 테스트 실행
check_vector_search(" kt에서 제공하는 티빙 스탠다드, MGC 커피구독을 해지하려면 결제일 기준 언제까지 해야 해?", top_k=3)

질문:  kt에서 제공하는 티빙 스탠다드, MGC 커피구독을 해지하려면 결제일 기준 언제까지 해야 해?

[Rank 1]
  - Score      : 0.8100
  - Content ID : KT_1665_policy_7
  - Text       :  이용료 결제일(서비스 이용기간 시작일) 기준 7일 이후에 해지하시는 경우에는, 즉시 해지 불가 및 해지 예약되고 이용요금은 해당월의 이용료가 전액 청구되며, 티빙 콘텐츠와 메가MGC커피 기프티쇼를 1달 동안 지속 이용 가능합니다. 예시) 최초 가입일이 1월 3일인 경우, 서비스 이용료 결제일은 매월 3일이며 매월 10일부터 서비스 이용료 결제일인 3일 이전까지 해지 시 '월 이용료 전액 부과'되며 익월 2일까지 티빙 콘텐츠와 메가MGC커피 기프티쇼 이용 가능 1) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시...
--------------------------------------------------
[Rank 2]
  - Score      : 0.8075
  - Content ID : KT_1579_policy_5
  - Text       : 련해서는 아래의 정책이 적용됩니다. 1) 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이내 해지하시는 경우에는, 티빙 콘텐츠와 스타벅스 기프티쇼 이용 여부에 따라 해지 처리됩니다. 가입 후 7일 이내 티빙 콘텐츠 및 스타벅스 기프티쇼 사용 이력이 없는 경우, 즉시 해지가 가능하며, 명세서에 청구되지 않습니다. 가입 후 7일 이내 티빙 콘텐츠 또는 스타벅스 기프티쇼 사용 이력이 존재하는 경우, 즉시 해지 불가 및 해지 예약되며 이용요금이 부과됩니다. 예시) 최초 가입일이 1월 3일인 경우, 서비...
--------------------------------------------------
[Rank 3]
  - Score      : 0.7995
  - Content ID : KT_1665_polic

### vector cypher

In [15]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

# [시나리오]
# 1. 벡터 검색(Vector Search)으로 질문과 의미가 가장 유사한 'Content(약관/정책)' 또는 'Benefit(혜택 상세)' 노드를 찾음.
# 2. 찾은 노드가 '약관(Policy)'이라면: 해당 약관을 보유한 'Product(상품)'를 역추적.
# 3. 찾은 노드가 '혜택(Benefit)'이라면: 해당 혜택을 제공하는 'Plan(요금제)'을 거쳐 상위 'Product(상품)'를 역추적.
# 4. 최종적으로 식별된 'Product'를 기준으로, 포함된 'Service(서비스)' 목록과 제공하는 모든 'Plan(요금제)'의 가격 정보를 함께 수집하여 반환.

retrieval_query = """
// 1. Vector 검색으로 찾은 node를 'content'라고 정의
WITH node AS content, score

// 2. 이 컨텐츠가 속한 상품(Product)을 찾음 (두 가지 경로 고려)
// 경로 A: Product -> HAS_POLICY -> Content (약관인 경우)
OPTIONAL MATCH (p1:Product)-[:HAS_POLICY]->(content)
// 경로 B: Product -> HAS_PLAN -> Plan -> OFFERS -> Content (혜택인 경우)
OPTIONAL MATCH (p2:Product)-[:HAS_PLAN]->(:Plan)-[:OFFERS]->(content)

WITH content, score, coalesce(p1, p2) AS product
WHERE product IS NOT NULL

// 3. 해당 상품의 서비스와 요금제 정보 수집
OPTIONAL MATCH (product)-[:INCLUDES_SERVICE]->(service:Service)
OPTIONAL MATCH (product)-[:HAS_PLAN]->(plan:Plan)

// 4. 결과 반환
RETURN
    content.text AS text,
    product.name AS product_name,
    product.url AS product_url,
    collect(DISTINCT service.name) AS included_services,
    collect(DISTINCT {
        plan_name: plan.title,
        price: plan.price_regular,
        discount_price: plan.price_kt
    }) AS related_plans,
    score
"""

vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name="vector_index",
    retrieval_query=retrieval_query,
    embedder=embedder
)

##  Text2Cypher

In [18]:
from neo4j_graphrag.retrievers import Text2CypherRetriever
def get_schema():
    with driver.session() as session:
        node_props = session.run("""
            CALL db.schema.nodeTypeProperties()
            YIELD nodeType, propertyName, propertyTypes
            WHERE propertyName IS NOT NULL
            RETURN nodeType, collect(propertyName) as properties
        """).data()

        rels = session.run("""
             MATCH (n)-[r]->(m)
             RETURN DISTINCT labels(n)[0] as source, type(r) as rel_type, labels(m)[0] as target
             LIMIT 50
         """).data()

        schema_text = "=== Graph Schema ===\n"

        schema_text += "[Nodes]\n"
        for n in node_props:
            schema_text += f"- {n['nodeType']} {n['properties']}\n"

        schema_text += "\n[Relationships]\n"
        for r in rels:
            schema_text += f"(:{r['source']})-[:{r['rel_type']}]->(:{r['target']})\n"

        return schema_text

# 스키마 확인
neo4j_schema = get_schema()
print(neo4j_schema)

=== Graph Schema ===
[Nodes]
- :`Product` ['product_id', 'name', 'url', 'fetched_at', 'promotion_info']
- :`Service` ['service_id', 'name', 'category']
- :`Plan` ['plan_id', 'title', 'price_regular', 'price_kt', 'price_promo', 'currency', 'billing_period']
- :`Content` ['content_id', 'name', 'detail', 'type', 'text', 'index', 'embedding', 'embedding_model', 'embedding_dim']
- :`BasePlan` ['key']
- :`Provider` ['provider_id', 'name']

[Relationships]
(:Provider)-[:PROVIDES]->(:Product)
(:Product)-[:INCLUDES_SERVICE]->(:Service)
(:Product)-[:HAS_PLAN]->(:Plan)
(:Product)-[:HAS_POLICY]->(:Content)
(:Plan)-[:BASED_ON]->(:BasePlan)
(:Plan)-[:OFFERS]->(:Content)



In [41]:
import os
from dotenv import load_dotenv
from neo4j_graphrag.llm import OpenAILLM

from google.colab import drive
drive.mount('/content/drive')

env_path = '/content/drive/MyDrive/project_Demo/start_final/.env'
load_dotenv(env_path)

groq_api_key = os.getenv("GROQ_API_KEY")

if not groq_api_key:
    raise ValueError(f"API Key를 찾을 수 없습니다. {env_path} 파일을 확인해주세요.")


os.environ["OPENAI_API_KEY"] = groq_api_key
os.environ["OPENAI_BASE_URL"] = "https://api.groq.com/openai/v1"
llm = OpenAILLM(
    model_name="llama-3.1-8b-instant", #llama-3.3-70b-versatile 무료 사용량 초과로 대체 모델 사용
    model_params={
        "temperature": 0,
        "max_tokens": 2000
    }
)

print(f" LLM 설정 완료: {llm.model_name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 LLM 설정 완료: llama-3.1-8b-instant


In [42]:
examples = [
    """
    USER INPUT: 티빙 프리미엄 요금제 가격 알려줘
    CYPHER QUERY:
    MATCH (p:Product {name: "TVING 이용권"})-[:OFFERS]->(plan:Plan)
    WHERE plan.plan_title CONTAINS "프리미엄"
    RETURN plan.plan_title, plan.prices
    """,

    """
    USER INPUT: 메가커피랑 같이 주는 상품의 약관 내용을 보여줘
    CYPHER QUERY:
    MATCH (p:Product)-[:HAS_CONTENT]->(c:Content)
    WHERE p.name CONTAINS "메가MGC커피"
    RETURN p.name, c.text
    LIMIT 1
    """,

    """
    USER INPUT: KT 구독 상품 중에 가장 비싼 요금제는 뭐야?
    CYPHER QUERY:
    MATCH (p:Product)-[:OFFERS]->(plan:Plan)
    RETURN p.name, plan.plan_title, plan.prices
    ORDER BY plan.prices DESC
    LIMIT 1
    """,

    """
    USER INPUT: 티빙 해지할 때 위약금 관련 정책 알려줘
    CYPHER QUERY:
    MATCH (c:Content)
    WHERE c.text CONTAINS "해지" OR c.text CONTAINS "위약금"
    RETURN c.text
    LIMIT 3
    """,
    """
    USER INPUT: 티빙 광고형 스탠다드 요금제 혜택이 뭐야?
    CYPHER QUERY:
    MATCH (plan:Plan)
    WHERE plan.plan_title CONTAINS '광고형 스탠다드'
    RETURN plan.plan_title, plan.benefits
    """,
    """
    USER INPUT: 티빙 이용권 프리미엄 요금제 가격 얼마야? KT 할인가도 알려줘.
    CYPHER QUERY:
    MATCH (p:Product)-[:OFFERS]->(plan:Plan)
    WHERE p.name CONTAINS 'TVING' AND plan.plan_title CONTAINS '프리미엄'
    RETURN p.name, plan.plan_title, plan.price_regular, plan.price_kt
    """
    """
    USER INPUT: 광고형 스탠다드 요금제는 화질이 어떻게 돼? 동시 접속은 몇 명까지야?
    CYPHER QUERY:
    MATCH (plan:Plan)-[:HAS_BENEFIT]->(b:Benefit)
    WHERE plan.plan_title CONTAINS '광고형 스탠다드' AND b.benefit_type = 'SPEC'
    RETURN plan.plan_title, b.max_quality, b.concurrent_streams
    """,
    """
    USER INPUT: 메가커피랑 같이 주는 상품 중에서 2만원 이하인 요금제 있어?
    CYPHER QUERY:
    MATCH (p:Product)-[:OFFERS]->(plan:Plan)
    WHERE p.name CONTAINS '메가MGC커피' AND plan.price_regular <= 20000
    RETURN p.name, plan.plan_title, plan.price_regular
    ORDER BY plan.price_regular ASC
    """,
    """
    USER INPUT: KT 구독 상품 해지하면 환불 규정이 어떻게 돼?
    CYPHER QUERY:
    MATCH (prov:Provider {name: 'KT구독'})-[:PROVIDES]->(p:Product)-[:HAS_CONTENT]->(c:Content)
    WHERE c.text CONTAINS '해지' OR c.text CONTAINS '환불'
    RETURN p.name, c.text
    LIMIT 3
    """,
    """
    USER INPUT: KT구독에서 제공하는 상품들은 어떤 서비스가 포함되어 있어?
    CYPHER QUERY:
    MATCH (prov:Provider {name: 'KT구독'})-[:PROVIDES]->(p:Product)-[:INCLUDES_SERVICE]->(s:Service)
    RETURN p.name AS Product, collect(s.name) AS Included_Services
    """
]

text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=neo4j_schema,
    examples=examples,
)

### 3가지 Retriever(Vector, VectorCypher, Text2Cypher)를 하나로 통합하여 ToolsRetriever와 GraphRAG를 구축

In [29]:
from neo4j_graphrag.retrievers import ToolsRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

# (1) Vector Retriever Tool: 단순 의미 검색
# - 상황: 사용자가  정책이나 약관 내용을 찾을 때
vector_tool = retriever.convert_to_tool(
    name="Policy_Vector_Search",
    description="사용자가 '해지', '환불', '유의사항' 등 약관이나 정책의 뉘앙스를 물어볼 때 사용합니다. 구체적인 수치보다는 텍스트의 의미를 찾을 때 적합합니다."
)

# (2) VectorCypher Retriever Tool: 문맥 + 구조적 정보(상품/요금제) 결합
# - 상황: 약관을 물어보면서 동시에 그 약관이 적용되는 상품명이나 요금제 리스트를 같이 알아야 할 때
vector_cypher_tool = vector_cypher_retriever.convert_to_tool(
    name="Comprehensive_Product_Search",
    description="사용자가 약관 내용과 함께, 해당 약관이 포함된 '상품 정보'와 '요금제 목록(가격 포함)'을 한꺼번에 알고 싶을 때 가장 유용합니다. (예: '커피 주는 요금제 설명해줘')"
)

# (3) Text2Cypher Retriever Tool: 정확한 사실/화질 등)을 콕 집어서 물어볼 때
text2cypher_tool = text2cypher_retriever.convert_to_tool(
    name="Factual_Data_Search",
    description="정확한 '가격', '요금제 이름', '혜택' 등 DB에 저장된 구체적인 사실이나 수치를 검색할 때 사용합니다. (예: '프리미엄 요금제 얼마야?')"
)
tools_retriever = ToolsRetriever(
    driver=driver,
    llm=llm,
    tools=[vector_tool, vector_cypher_tool, text2cypher_tool]
)

print("[OK] ToolsRetriever 설정 완료")

[OK] ToolsRetriever 설정 완료


In [36]:

prompt_template = RagTemplate(
    template="""당신은 구독 상품 정보를 제공하는 전문 상담 AI입니다.

질문: {query_text}

검색된 정보(Context):
{context}

지침:
1. 제공된 검색 결과(Context)가 JSON 형식이거나 리스트일 수 있습니다. 내용을 잘 파석하여 답변하세요.
2. 답변에는 반드시 **상품명(Product Name)**, **관련 요금제(Plan)**, **가격(Price)** 정보를 포함하려고 노력하세요.
3. 약관이나 주의사항이 있다면 핵심 내용을 요약해서 전달하세요.
4. 검색 결과에 없는 내용은 "죄송하지만 해당 정보는 찾을 수 없습니다"라고 말하고 추측하지 마세요.
5. 금액은 '원' 단위로 명확히 표기하세요.

답변 형식 예시:
- [상품명] (상품 URL)
  주요 요금제: [요금제 이름] - [가격]원
  관련 정책/내용: [내용 요약]

답변:""",
    expected_inputs=["context", "query_text"]
)

# ---------------------------------------------------------
# 4. GraphRAG 생성
# ---------------------------------------------------------
graphrag = GraphRAG(
    llm=llm,
    retriever=tools_retriever, # 여기서 ToolsRetriever를 연결
    prompt_template=prompt_template
)

print("[OK] GraphRAG(Agent) 준비 완료")

[OK] GraphRAG(Agent) 준비 완료


In [ ]:
# GraphRAG 대화형 인터페이스

print(" 'exit', 'quit', 'q'를 입력하면 종료됩니다.")

while True:
    user_query = input("\n[질문 입력]: ").strip()

    if user_query.lower() in ['exit', 'quit', 'q']:
        print("\n챗봇을 종료합니다.")
        break

    # 빈 입력 방지
    if not user_query:
        continue

    print("\n 검색 및 답변 생성 중...")

    try:
        result = graphrag.search(query_text=user_query, return_context=True)

        print(f" [질문]: {user_query}")
        print("\n" + "="*50)
        print("[답변]")
        print(result.answer)
        print("="*50)


    except Exception as e:
        print(f"\n 오류가 발생했습니다: {e}")

💡 'exit', 'quit', 'q'를 입력하면 종료됩니다.

 검색 및 답변 생성 중...
 [질문]: "티빙 프리미엄 가격은 얼마고 화질은 어때?"

[답변]
티빙 프리미엄 (https://www.tving.com/)
  주요 요금제: 티빙 프리미엄 - 13,900원
  관련 정책/내용: 티빙 프리미엄은 다양한 콘텐츠를 고화질(1080p)로 시청할 수 있습니다.

 검색 및 답변 생성 중...
 [질문]: 티빙이용권을 가입하고 나서 한 번도 안 썼는데 취소될까?

[답변]
- TVING+메가MGV커피 
  주요 요금제: 프리미엄+메가MGC커피, 스탠다드+메가MGC커피, 베이직+메가MGC커피, 광고형스탠다드+메가MGC커피 
  관련 정책/내용: 가입 후 7일 이내 티빙 콘텐츠 및 메가MGC커피 기프티쇼 사용 이력이 없는 경우, 즉시 해지가 가능하며, 명세서에 청구되지 않습니다. 티빙 이용권을 가입하고 나서 한 번도 안 쓴 경우, 즉시 해지가 가능하며, 명세서에 청구되지 않습니다.

 검색 및 답변 생성 중...
 [질문]: 메가커피 주는 상품 해지하면 위약금 있어?

[답변]
- TVING+메가MGV커피 
  주요 요금제: 프리미엄+메가MGC커피, 스탠다드+메가MGC커피, 베이직+메가MGC커피, 광고형스탠다드+메가MGC커피 - 가격 정보는 제공된 검색 결과에 없습니다.
  관련 정책/내용: 부가서비스 가입 후, 매월 서비스 이용료 결제일(서비스 이용기간 시작일) 기준 7일 이내 해지 시, 티빙 콘텐츠와 메가MGC커피 기프티쇼 이용 여부에 따라 해지 처리됩니다. 가입 후 7일 이내 티빙 콘텐츠 및 메가MGC커피 기프티쇼 사용 이력이 없는 경우, 즉시 해지가 가능하며, 명세서에 청구되지 않습니다. 가입 후 7일 이후에 해지하는 경우, 즉시 해지 불가 및 해지 예약되고 이용요금은 해당월의 이용료가 전액 청구되며, 티빙 콘텐츠와 메가MGC커피 기프티쇼를 1달 동안 지속 이용 가능합니다. 위약금에 대한 정보는 제공된 검색 결과에 없습니다.

 검색 및 답변 생성 중

Retrying neo4j_graphrag.utils.rate_limit.rate_limit_handler.<locals>.wrapper.<locals>.inner_func in 1.8323264033651583 seconds as it raised RateLimitError: Rate limit exceeded: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kc9ckaqven5vagd2w7y9ktm6` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99934, Requested 1069. Please try again in 14m26.592s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.
Retrying neo4j_graphrag.utils.rate_limit.rate_limit_handler.<locals>.wrapper.<locals>.inner_func in 1.1191618465908795 seconds as it raised RateLimitError: Rate limit exceeded: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kc9ckaqven5vagd2w7y9ktm6` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99932, Requested 1069


❌ 오류가 발생했습니다: RetryError[<Future at 0x27e0429b970 state=finished raised RateLimitError>]

챗봇을 종료합니다.


In [ ]:
driver.close()